In [9]:
import re
import csv 
import json
import pymysql
import pymysql.cursors
import mysql.connector
from getpass import getpass 
from selenium import webdriver
from time import sleep 
from selenium.webdriver.common.keys import Keys 
from selenium.common.exceptions import NoSuchElementException 
from msedge.selenium_tools import Edge, EdgeOptions 
from kafka import KafkaProducer
from kafka.errors import KafkaError
producer = KafkaProducer(bootstrap_servers=['localhost:9092'])

In [10]:
options = EdgeOptions()
options.use_chromium = True 
driver = Edge(executable_path=r'C:\msedgedriver.exe', options=options)
driver.get('https://twitter.com/i/flow/login')

C:\Users\asus\AppData\Local\Temp/ipykernel_20796/2047074698.py:3: DeprecationWarning: Selenium Tools for Microsoft Edge is deprecated. Please upgrade to Selenium 4 which has built-in support for Microsoft Edge (Chromium): https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/#upgrading-from-selenium-3
  driver = Edge(executable_path=r'C:\msedgedriver.exe', options=options)


In [11]:
user=driver.find_element_by_xpath('//input[@class="r-30o5oe r-1niwhzg r-17gur6a r-1yadl64 r-deolkf r-homxoj r-poiln3 r-7cikom r-1ny4l3l r-t60dpp r-1dz5y72 r-fdjqy7 r-13qz1uu"]')
username = getpass('username : ') 
user.send_keys(username) 
user.send_keys(Keys.RETURN)

C:\Users\asus\AppData\Local\Temp/ipykernel_20796/4162907471.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  user=driver.find_element_by_xpath('//input[@class="r-30o5oe r-1niwhzg r-17gur6a r-1yadl64 r-deolkf r-homxoj r-poiln3 r-7cikom r-1ny4l3l r-t60dpp r-1dz5y72 r-fdjqy7 r-13qz1uu"]')


In [12]:
passfield = driver.find_element_by_xpath('//input[@name="password"]')
password = getpass('Password: ') 
passfield.send_keys(password)
passfield.send_keys(Keys.RETURN)

C:\Users\asus\AppData\Local\Temp/ipykernel_20796/172388547.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  passfield = driver.find_element_by_xpath('//input[@name="password"]')


Password: ········


In [13]:
driver.find_element_by_xpath('//a[@aria-label="Search and explore"]').click()

C:\Users\asus\AppData\Local\Temp/ipykernel_20796/3051703618.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_xpath('//a[@aria-label="Search and explore"]').click()


In [14]:
search=driver.find_element_by_xpath('//input[@aria-label="Search query"]')
search.send_keys('budget2022') 
search.send_keys(Keys.RETURN)
driver.find_element_by_link_text('Latest').click()

C:\Users\asus\AppData\Local\Temp/ipykernel_20796/3629777371.py:1: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  search=driver.find_element_by_xpath('//input[@aria-label="Search query"]')
C:\Users\asus\AppData\Local\Temp/ipykernel_20796/3629777371.py:4: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  driver.find_element_by_link_text('Latest').click()


<h2>Storing Tweets in kafka</h2>

In [ ]:
page_cards = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')
for card in page_cards[0:]:
    text = card.find_element_by_xpath('.//div[@class="css-901oao r-1fmj7o5 r-37j5jr r-a023e6 r-16dba41 r-rjixqe r-bcqeeo r-bnwqim r-qvutc0"]').text 
    x =text.encode("utf-8")
    producer.send('project', x)

<h2>Storing Tweet in MySQL</h2>

In [ ]:
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  password="<password>",
    database = 'twitter'
)

page_cards = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')
for card in page_cards[0:]:
    text = card.find_element_by_xpath('.//div[@class="css-901oao r-1fmj7o5 r-37j5jr r-a023e6 r-16dba41 r-rjixqe r-bcqeeo r-bnwqim r-qvutc0"]').text
    x=text.replace ('\n', '')
    print(x)
    cursorObject = mydb.cursor()
    sql = "INSERT INTO twee""(tw)""VALUES(%s)"
    cursorObject.execute("INSERT INTO twee (tw) VALUES (%s)", (x,))
    mydb.commit()
   
# disconnecting from server
mydb.close()

<h2>Storing Tweet in Singlestore</h2>

In [ ]:
mydb =pymysql.connect(
    user='root',
    password='root',
    host='localhost',
    port=3307,
    database='<database name>')

page_cards = driver.find_elements_by_xpath('//article[@data-testid="tweet"]')
for card in page_cards[0:]:
    text = card.find_element_by_xpath('.//div[@class="css-901oao r-1fmj7o5 r-37j5jr r-a023e6 r-16dba41 r-rjixqe r-bcqeeo r-bnwqim r-qvutc0"]').text
    x=text.replace ('\n', '')
    print(x)
    cursorObject = mydb.cursor()
    sql = "INSERT INTO word""(twe)""VALUES(%s)"
    cursorObject.execute(sql, (x,))
    mydb.commit()
   
# disconnecting from server
mydb.close()